In [1]:
import glob
from pathlib import Path

In [6]:
sorted([x for x in Path("data/align_image_1_3").glob("*/")])

[PosixPath('data/align_image_1_3/.DS_Store'),
 PosixPath('data/align_image_1_3/1'),
 PosixPath('data/align_image_1_3/10'),
 PosixPath('data/align_image_1_3/100'),
 PosixPath('data/align_image_1_3/101'),
 PosixPath('data/align_image_1_3/102'),
 PosixPath('data/align_image_1_3/103'),
 PosixPath('data/align_image_1_3/104'),
 PosixPath('data/align_image_1_3/105'),
 PosixPath('data/align_image_1_3/106'),
 PosixPath('data/align_image_1_3/107'),
 PosixPath('data/align_image_1_3/108'),
 PosixPath('data/align_image_1_3/109'),
 PosixPath('data/align_image_1_3/11'),
 PosixPath('data/align_image_1_3/110'),
 PosixPath('data/align_image_1_3/111'),
 PosixPath('data/align_image_1_3/112'),
 PosixPath('data/align_image_1_3/113'),
 PosixPath('data/align_image_1_3/114'),
 PosixPath('data/align_image_1_3/115'),
 PosixPath('data/align_image_1_3/116'),
 PosixPath('data/align_image_1_3/117'),
 PosixPath('data/align_image_1_3/118'),
 PosixPath('data/align_image_1_3/119'),
 PosixPath('data/align_image_1_3/12'),